# Please run the cells by pressing Shift + Enter
First import the necesseary libraries and the napari viewer.

In [1]:
import numpy as np
import vigra
import napari
import matplotlib
import matplotlib.pyplot as plt
import pickle
import cv2 as cv
import geopandas as gpd
import pandas
import os
import json
import h5py
import tifffile
import orthogonality_map
import AnomalyEdge
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, jaccard_score, f1_score
from skimage.measure import label, regionprops
from skimage import filters, morphology, transform as transf
from scipy import ndimage as ndi, signal
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from shapely.geometry import Polygon, MultiPolygon
from shapely.affinity import translate, scale, affine_transform
from mlxtend.plotting import plot_confusion_matrix

In [2]:
# Set up the GUI and start the napari viewer
%matplotlib qt
%gui qt
viewer = napari.Viewer()

# 1. Import geophysical image 
Import the geophysical image to be analysed into the napari viewer using 'File -> Open File(s)'. If you would like to open a 3D dataset consisting of several horizontal slices, use the option: 'File -> Open Files as Stack'. This will allow you to select all horizontal slice images at once, and import the complete volume in napari. When importing the data volume in this way, an error message may appear when calculating the features in step 2.2. To avoid this, save the data volume as a 3D TIFF file by selecting 'File -> Save Selected Layer(s)', then remove the layer with the data volume from the layer list, and re-open the saved 3D TIFF file. 3D data can be visualised as horizontal slices, vertical slices (xz and yz plane) and as a volume; use the second and third button in the lower left corner of the viewer.<br> Run the code below to import the input data as a numpy array, extract the path of the input data and determine the number of dimensions for late use.

In [3]:
image = viewer.layers[0].data
path = viewer.layers[0].source.path
nrofdim = image.ndim

# 2. Machine learning based pixel classification
Pixel classification using the code below will usually be slower than in segmentation tools such as ilastik, LABKIT or Trainable Weka Segmentation. Usually, for 2D data the difference in speed is not very big. For 3D data, run times may be significantly longer and RAM usage may be significantly larger. If pixel segmentation has been performed by means of an external tool such as ilastik, please import the segmented image in step 3.1 below.

## 2.1. Load features
If no pixel features have been calculated for your input data, please go to step 2.2.<br>
If pixel features, feature names and scales for the input data have previously been saved, they can be loaded here. Please adjust the file name in the first line of each cell if necessary. The files should be in the same folder as the Jupyter Notebook, or their path should be specified in the code below. After loading the features, please go immediately to step 2.4.

In [4]:
f = open('Features.pckl', 'rb')
features = pickle.load(f)
f.close()
X = features.T

In [5]:
f = open('Feature_names.pckl', 'rb')
feature_names = pickle.load(f)
f.close()

In [6]:
f = open('Feature_scales.pckl', 'rb')
feature_scales = pickle.load(f)
f.close()

## 2.2. Select scales and features
The feature scales below will be used for the pixel classification. These scales are defined by the standard deviation (sigma) of the kernel used for the gaussian smoothing which is carried out before the caluclation of the features. The scales (sigmas) below are appropriate for most geophysical datasets. You can add or remove scales by modifying the code line below. If you use a pretrained classifier, please use the same scales and features as were previously used for training the classifier.<br> If an error message appears when calculating the features saying that 'the kernel is longer than the line', this may indicate that one or more selected feature scales are too large for the data. This can happen for example in the case of 3D data, when the kernel (which is then also 3D) becomes too large in comparison with the number of horizontal slices. In that case, please remove the largest scale(s).

In [17]:
sigmas = [0.7, 1.0, 1.6, 3.5, 5.0, 10.0]

The image features below will be used for the pixel classification. These are appropriate for most geophysical datasets. You can deselect features by deleting them or commenting them out by putting a '#' at the beginning of the line. Other features can be added and appended to the feature stack. Unless training and segmentation (section 2.6) are slow, it is recommended to use all image features below. 

In [18]:
imageFloat = image.astype('float64')
feature_stack = []
feature_names = []
feature_scales = []

# Gaussian smoothing with sigma = 0.3
gaussian_smoothing = vigra.filters.gaussianSmoothing(imageFloat, 0.3)
feature_stack.append(gaussian_smoothing.ravel())
feature_names.append('Gaussian smoothing')
feature_scales.append(0.3)

for sigma in sigmas:   
    # Gaussian smoothing
    gaussian_smoothing = vigra.filters.gaussianSmoothing(imageFloat, sigma)
    feature_stack.append(gaussian_smoothing.ravel()) 
    feature_names.append('Gaussian smoothing')
    feature_scales.append(sigma)

    # Laplacian of Gaussian
    laplacian_of_gaussian = vigra.filters.laplacianOfGaussian(imageFloat, sigma)
    feature_stack.append(laplacian_of_gaussian.ravel()) 
    feature_names.append('Laplacian of Gaussian')
    feature_scales.append(sigma)

    # Gaussian Gradient Magnitude
    gaussian_gradient_magnitude = vigra.filters.gaussianGradientMagnitude(imageFloat, sigma)
    feature_stack.append(gaussian_gradient_magnitude.ravel()) 
    feature_names.append('Gaussian gradient magnitude')
    feature_scales.append(sigma)

    # Difference of Gaussians
    difference_of_gaussians = vigra.filters.gaussianSmoothing(imageFloat, sigma) - vigra.filters.gaussianSmoothing(imageFloat, sigma * 0.66)
    feature_stack.append(difference_of_gaussians.ravel())
    feature_names.append('Difference of Gaussians')
    feature_scales.append(sigma)

    # Stucture tensor eigenvalues
    structure_tensor_eigenvalues = vigra.filters.structureTensorEigenvalues(imageFloat, sigma, sigma * 0.5)[...,0]
    feature_stack.append(structure_tensor_eigenvalues.ravel())
    feature_names.append('Structure tensor eigenvalues')
    feature_scales.append(sigma)

    #Hessian of Gaussian eigenvalues
    hessian_of_gaussian_eigenvalues = vigra.filters.hessianOfGaussianEigenvalues(imageFloat, sigma)[...,1]    
    feature_stack.append(hessian_of_gaussian_eigenvalues.ravel())    
    feature_names.append('Hessian of Gaussian eigenvalues')
    feature_scales.append(sigma)
features = np.asarray(feature_stack)
X = features.T

## 2.3. Save features
If further classification tests will be carried out on the same input data in the future, it may be useful to store pixel features,  feature names and scales for later use. Please adjust the file names or path names in the first line of each cell if necessary (in the code below, files are saved in the same folder as the Jupyter Notebook). 

In [19]:
f = open('Features.pckl', 'wb')
pickle.dump(features, f)
f.close()

In [20]:
f = open('Feature_names.pckl', 'wb')
pickle.dump(feature_names, f)
f.close()

In [21]:
f = open('Feature_scales.pckl', 'wb')
pickle.dump(feature_scales, f)
f.close()

## 2.4. View features

You can create an overview table of the features and scales by running the code below.

In [22]:
df = pandas.DataFrame(list(zip(feature_names, feature_scales)), columns = ['Feature name', 'Scale'])
print(df)

                       Feature name  Scale
0                Gaussian smoothing    0.3
1                Gaussian smoothing    0.7
2             Laplacian of Gaussian    0.7
3       Gaussian gradient magnitude    0.7
4           Difference of Gaussians    0.7
5      Structure tensor eigenvalues    0.7
6   Hessian of Gaussian eigenvalues    0.7
7                Gaussian smoothing    1.0
8             Laplacian of Gaussian    1.0
9       Gaussian gradient magnitude    1.0
10          Difference of Gaussians    1.0
11     Structure tensor eigenvalues    1.0
12  Hessian of Gaussian eigenvalues    1.0
13               Gaussian smoothing    1.6
14            Laplacian of Gaussian    1.6
15      Gaussian gradient magnitude    1.6
16          Difference of Gaussians    1.6
17     Structure tensor eigenvalues    1.6
18  Hessian of Gaussian eigenvalues    1.6
19               Gaussian smoothing    3.5
20            Laplacian of Gaussian    3.5
21      Gaussian gradient magnitude    3.5
22         

You can view a feature by entering its number in the first line of the cell below. The number can be found by looking up the feature name and scale in the table above. If the data is 3D, please also enter the horizontal slice to be viewed. The image with the selected feature will open in a separate window.

In [24]:
feature_index = 20 # Select the feature to be viewed from the table above
selectedSlice = 48 # For 3D data, select the slice to be viewed

# Select feature
selectedFeature = features[feature_index,:] 
if nrofdim == 2:
    selectedFeatureReshaped = selectedFeature.reshape(image.shape)
else:    
    slice_size = np.prod(image.shape[1:3])
    selectedFeatureSlice = selectedFeature[(selectedSlice * slice_size):((selectedSlice + 1) * slice_size)]
    selectedFeatureReshaped = selectedFeatureSlice.reshape(image.shape[1:3])

# Clip values to enhance visualisation
mean_val = selectedFeatureReshaped.mean()
std_val = selectedFeatureReshaped.std()
clip_min, clip_max = mean_val - (3 * std_val), mean_val + (3 * std_val)
selectedFeatureClip = np.clip(selectedFeatureReshaped, clip_min, clip_max)

# Display feature
plt.imshow(selectedFeatureClip,cmap='gray')

## 2.5. Load classifier
If a classifier has been previously trained and saved, it can be loaded here. Please specify its name in the code below. It should be in the same folder as the Jupyter Notebook, or its path should be specified in the code below. After loading the classifier, please go to step 2.6.2 to apply it to the input data.

In [8]:
f = open('Pixel_classifier.pckl', 'rb')
classifier = pickle.load(f)
f.close()

## 2.6. Iterative training and segmentation
### 2.6.1. Training
If no classifier has been loaded, please first run the code below if a file with labels (in H5 format) is available (for example if the data have been labeled in other software such as ilastik). Please adjust the file name in the first line if necessary. The file should be in the same folder as the Jupyter Notebook, or its path should be specified in the code below. 

In [5]:
# Open H5 file with training labels and add it to the napari viewer
filename = "Labels.h5"
with h5py.File(filename, "r") as f:    
    key = list(f.keys())[0]     
    training_labels = f[key][...,0]
viewer.add_labels(training_labels)

<Labels layer 'training_labels [1]' at 0x22006a6b2b0>

If no file with labels is available, please add an empty 'training_labels' layer to napari by running the cell below.

In [4]:
training_labels = np.zeros_like(image, dtype=np.uint8)
viewer.add_labels(training_labels)

<Labels layer 'training_labels' at 0x220034c7250>

For the iterative training and classification, please follow the procedure below.
1. Unless labels have been imported (see the first cell above), train the classifier by selecting the 'training_labels' layer in napari and adding labels to it, using the paint brush and other instruments (see the icons in the top left corner of the viewer under 'Layer controls'). This notebook requires two classes at the stage of pixel classification: background (= label 1) and archaeological structures (= label 2). Further classification occurs at the object level (see Section 3). Nevertheless, in theory it is possible to add more classes at this stage.  
2. When the labels have been added, run the cells below to train the classifier and carry out the segmentation (step 2.6.2). After the segmentation has been completed, the result of the segmentation is automatically added as the 'segmentation' layer in napari. 
3. Additional labels can then be added in napari (in the 'training_labels' layer), and training and segmentation can be carried out again by running the steps below, in an iterative way, until the result of the segmentation is satisfactory.
4. The labels can be saved by selecting the 'training_labels' layer and applying 'File -> Save Selected Layer(s)' in napari.

In [28]:
# Remove 'segmentation' layer if it exists
if 'segmentation' in viewer.layers:
    viewer.layers.remove('segmentation')

# Get and preprocess training labels
training_labels = viewer.layers['training_labels'].data.ravel()
mask = training_labels > 0
Xsel, ysel = X[mask], training_labels[mask]

# Train the classifier
classifier = RandomForestClassifier(n_jobs=-1)
classifier.fit(Xsel, ysel)

RandomForestClassifier(n_jobs=-1)

### 2.6.2. Segmentation
Please run the code below to apply the trained classifier to the input data. The results of the segmentation are added as the 'segmentation' layer in napari. The segmentation is obtained by assigning to each pixel the class label with the highest probability.

In [29]:
result = classifier.predict(X)
segmentation = result.reshape(image.shape)
viewer.add_labels(segmentation, opacity = 0.4)

<Labels layer 'segmentation' at 0x1b37ca35fd0>

## 2.7. Save classifier
To save the classifier, please specify its name below. In the default code below, it is saved in the same folder as the Jupyter Notebook. Otherwise, its path should be specified. 

In [30]:
f = open('Pixel_classifier.pckl', 'wb')
pickle.dump(classifier, f)
f.close()

## 2.8. Apply hysteresis threshold and create objects
### 2.8.1. Calculation of probabilities

In the segmentation in step 2.6.2, the class with the highest probability is assigned to each pixel. To perform a more refined segmentation, hysteresis thresholding can be applied to the probability map of a certain class (usually the 'archaeological structures' class). To do that, probabilities need to be calculated first.

In [31]:
probabilities = classifier.predict_proba(X)

The code below adds the probability maps to the napari viewer (for the two classes 'background' and 'archaeological structures' as described in section 2.6.1).

In [32]:
# Probability map for the first class ('background')
probabilitymap = probabilities[:,0].reshape(image.shape)
viewer.add_image(probabilitymap, opacity = 1.0, name = "Probability (class 1)")

# Probability map for the second class ('archaeological structures')
probabilitymap = probabilities[:,1].reshape(image.shape)
viewer.add_image(probabilitymap, opacity = 1.0, name = "Probability (class 2)")

<Image layer 'Probability (class 2)' at 0x1b3742dde20>

### 2.8.2. Hysteresis thresholding
For the application of hysteresis thresholding, there are two thresholds. Pixels that belong to the 'archaeological structures' class with a probability above the low threshold are only selected if they are connected to pixels above a higher, more stringent probability threshold (the high threshold). Both thresholds can be set in the code below.

In [33]:
# Reshape the probability map so that it has the same dimensions as the geophysical image
reshaped_probabilities = probabilities[:,1].reshape(image.shape)

# Apply gaussian smoothing (the standard devitation of the gaussian kernel can be adjusted)
smoothed_probabilities = vigra.filters.gaussianSmoothing(reshaped_probabilities, 0.5)

# Hyseresis thresholding
low = 0.5     # Enter the low threshold
high = 0.6   # Enter the high threshold
hyst = filters.apply_hysteresis_threshold(smoothed_probabilities, low, high)

# Add the thresholded data to the napari viewer
viewer.add_labels(hyst, name = "Segmentation after hysteresis thresholding (Class 2)")

<Labels layer 'Segmentation after hysteresis thresholding (Class 2)' at 0x1b3018b3a60>

### 2.8.3. Creation of individual objects
Regions are labelled so that they can be used for subsequent object classification. Pixels receive the same label if they are neighbours in horizontal, vertical or diagonal direction. Connectivity can be changed to '1' if regions with the same label should only consist of pixels connected in horizontal or vertical (and not in diagonal) direction.

In [34]:
labeled_hyst = label(hyst, connectivity = 2, background = 0)
viewer.add_labels(labeled_hyst, name = "Objects (Class 2)")

<Labels layer 'Objects (Class 2)' at 0x1b37fcb4400>

# 3. Object classification
## 3.1. Import .h5 file with segmented objects
If the pixel classification has been done in another software package (e.g. ilastik), please import the H5 file with the segmentation by running the code below. Please adjust the file name in the first line if necessary. The file should be in the same folder as the Jupyter Notebook, or its path should be specified in the code below. 

In [6]:
# Open H5 file with labeled objects and add it to the napari viewer
filename = "Binary_image.h5"
with h5py.File(filename, "r") as f:    
    key = list(f.keys())[0]     
    hyst = f[key][...,0]         
labeled_hyst = label(hyst)
viewer.add_labels(labeled_hyst, name = "Segmentation")

<Labels layer 'Segmentation' at 0x22006c3c790>

## 3.2. Watershed segmentation

Often, the objects are too big for straightforward classification, because they encompass pixels belonging to more than one semantic class (for example parts of a wall and a floor). Watershed is applied to reduce the size of the regions and make them more semantically homogeneous. The size of the resulting regions can be controlled by the size of the footprint in the code below. If no watershed is to be used, please go immediately to the last cell of this section.<br>
The code is based on: https://scikit-image.org/docs/stable/auto_examples/segmentation/plot_watershed.html

In [35]:
# Convert the labeled objects to 'int32' format 
labeled_hyst = labeled_hyst.astype(np.int32)

# Compute the distance transform
distance = ndi.distance_transform_edt(labeled_hyst.astype(np.int32))

# Determine the footprint size (number of pixels or voxels)
footprint_size = (30, 30) if nrofdim == 2 else (20, 20, 20)

# Find local maxima
coords = peak_local_max(distance, footprint=np.ones(footprint_size), labels=labeled_hyst)

# Create a mask for the markers
mask = np.zeros_like(distance, dtype=bool)
mask[tuple(coords.T)] = True

# Label the markers
markers, _ = ndi.label(mask)

# Apply watershed
labeled_hyst_watershed = watershed(-distance, markers, mask=labeled_hyst)

# Add the result to the viewer
viewer.add_labels(labeled_hyst_watershed, opacity = 0.8)

<Labels layer 'labeled_hyst_watershed' at 0x1b37fcbd310>

If no watershed is used, please run the code below:

In [5]:
labeled_hyst_watershed = labeled_hyst

## 3.3. Machine learning based object classification
If only manual object classification will be carried out, please go directly to section 4.
## 3.3.1. Selection of object features
First run the code below to calculate properties of the objects created in section 2.8 or imported in section 3.1 (possibly after watershed segmentation; section 3.2) and create the object feature stack.

In [6]:
stats = regionprops(labeled_hyst_watershed, intensity_image=image)
object_feature_stack = []

### 3.3.1.1. Standard object features in scikit-image

A large number of object features exist, see the list in https://scikit-image.org/docs/stable/api/skimage.measure.html#skimage.measure.regionprops.<br> Not all features are supported for 3D data. If you are analysing 3D data, please run the code below for lists of supported and unsupported 3-D features (the code is based on: https://scikit-image.org/skimage-tutorials/lectures/three_dimensional_image_processing.html)

In [ ]:
supported = [] 
unsupported = []

for s in stats[0]:
    try:
        stats[0][s]
        supported.append(s)
    except NotImplementedError:
        unsupported.append(s)

print("Supported properties:")
print("  " + "\n  ".join(supported))
print()
print("Unsupported properties:")
print("  " + "\n  ".join(unsupported))

A few combinations of object features are given below (for magnetometer and for GPR). Object features can be appended to the feature stack by adding a code line of the same form and including the name of the feature from the list created in the previous step or found on https://scikit-image.org/docs/stable/api/skimage.measure.html#skimage.measure.regionprops.

In [7]:
# Magnetometer data
object_feature_stack.append(np.asarray([s.area for s in stats]))
object_feature_stack.append(np.asarray([s.eccentricity for s in stats]))
object_feature_stack.append(np.asarray([s.solidity for s in stats]))

In [ ]:
# GPR data
#object_feature_stack.append(np.asarray([s.eccentricity for s in stats]))
#object_feature_stack.append(np.asarray([s.orientation for s in stats]))
object_feature_stack.append(np.asarray([s.solidity for s in stats]))

In [6]:
object_feature_stack.append(np.asarray([s.area for s in stats]))
object_feature_stack.append(np.asarray([s.centroid[0] for s in stats]))

### 3.3.1.2. Combinations of standard object features

New object features can be created by combining standard features included in scikit-image. The examples below create new features from combinations of standard features, and append the newly created feature to the feature stack.

In [8]:
# The following code can be used to classify small dipoles in magnetometer data

# Extract areas, intensity_min and intensity_max from stats
areas = np.asarray([s.area for s in stats])
intensity_min = np.asarray([s.intensity_min for s in stats])
intensity_max = np.asarray([s.intensity_max for s in stats])

# Calculate intensity difference and ratio
ratio_intensity_size = (intensity_max - intensity_min) / areas

# Append to object_feature_stack
object_feature_stack.append(ratio_intensity_size)

In [7]:
# Creation of a 'roundness' feature
roundness = np.asarray([s.equivalent_diameter_area / s.axis_major_length for s in stats])
object_feature_stack.append(roundness)

### 3.3.1.3. New object features

It is also possible to create entirely new object features. The code below shows the example of two features than can be useful to classify objects belonging to linear structures having two dominant perpendicular orientations, for example walls. The following steps are required to create these object features.
1. In the first code cell below, define the size of the rectangular structuring element (SE) which is representative for the linear structures, taking into account pixel size (e.g. if the distance between pixels is 0.05 m in x- and y-direction, a SE of 60 x 5 pixels equals 3 m x 0.25 m).
2. Find the dominant orientation of the linear structures in the geophysical map. This orientation is found automatically by the algorithm using the Hough transform. For 3D volumes, please indicate in the first cell below which horizontal slice should be used for the calculation of the orientation (choose a slice where the structures are clear; the slice number can be seen in the bottom right corner of the napari viewer). For 2D images, indicate '0' as slice number.
3. Calculate the orthogonality map, which shows to what extent each pixel belongs to a linear structure following the dominant orientation. The algorithm uses the rank-max opening (see the paper and supplementary data accompanying it for more details). Usually the 'rank' parameter in the code cell below can be left unchanged. The calculation is slow (it may take several hours for a 3D volume with many depth slices). The orthogonality map can be saved (and loaded again) for future use, by means of the second and third cells below. 
4. From the orthognality map, two object features ('mean orthogonality' and 'maximum orthogonality') are derived in the fourth code cell below, and appended to the feature stack.

In [8]:
# Define the size of structuring element, the horizontal slice of the geophyiscal volume that should be used for deriving 
# the dominant orientation of the archaeological structures, and the rank to be used by the rank-max opening 

# Structuring element
length = 60
width = 5

# Horizontal slice (this parameter is only used for 3D data)
slicenr = 40 

# Rank
rank = 75 # mostly this can be left unchanged; if necessary, change this value to experiment with different ranks

# Calculate orthogonality map
orthogonalityMap = orthogonality_map.orthogonality_map(image, labeled_hyst_watershed, length, width, slicenr, rank, nrofdim)

In [9]:
f = open('Orthogonality_map.pckl', 'wb')
pickle.dump(orthogonalityMap, f)
f.close()

In [5]:
f = open('Orthogonality_map.pckl', 'rb')
orthogonalityMap = pickle.load(f)
f.close()

In [6]:
viewer.add_image(orthogonalityMap)

<Image layer 'orthogonalityMap' at 0x1bd65c8e5e0>

In [9]:
# Calculate properties of the ojects (the properties related to the intensity are based on the orthogonality map)
stats_orthogonality = regionprops(labeled_hyst_watershed, intensity_image=orthogonalityMap)

# Derive object features
object_feature_stack.append(np.asarray([s.intensity_mean for s in stats_orthogonality]))  
object_feature_stack.append(np.asarray([s.intensity_max for s in stats_orthogonality]))

### 3.3.1.4. Put the object feature stack in the right format so that it can be used for training the classifier.

In [9]:
object_features = np.asarray(object_feature_stack)
Xobj = object_features.T
Xobj[Xobj == np.inf] = 0

## 3.3.2. Load classifier
If an object classifier has been previously trained and saved, it can be loaded here. Please specify its name in the code below. It should be in the same folder as the Jupyter Notebook, or its path should be specified in the code below. After loading the classifier, please go immediately to step 3.3.3.4.

In [11]:
f = open('Object_classifier.pckl', 'rb')
classifierObj = pickle.load(f)
f.close()

## 3.3.3. Interactive training and classification
If no classifier has been loaded, please first run the code below to add an empty 'training_labels_object' layer to napari. Afterwards, if you would like to use raster labels for the classification, please go to step 3.3.3.1; if you would like to use vector shapes, please go to step 3.3.3.2.

In [7]:
training_labels_object = np.zeros(image.shape[:3], dtype=np.uint8)
viewer.add_labels(training_labels_object)

<Labels layer 'training_labels_object' at 0x220071ecf70>

### 3.3.3.1. Creating training data by means of raster labels
For the iterative training and classification, please carry out the procedure below.
1. Train the classifier by selecting the 'training_labels_object' layer in napari and adding labels to it, and do so for as many classes as needed, using the paint brush and other instruments (see the icons in the top left corner of the viewer under 'Layer controls'). To label an object, it is sufficient that at least one pixel belonging to the object is labeled. If an object has received different labels, the label with the highest class number prevails.
2. When the labels have been added, run the code below and then go to steps 3.3.3.3 and 3.3.3.4. 
3. After running the code in 3.3.3.3 and 3.3.3.4, additional labels can be added in napari (in the 'training_labels_object' layer), and classification can be carried out again by running the code line below and in steps 3.3.3.3 and 3.3.3.4, in an iterative way, until the result of the classification is satisfactory.

In [10]:
training_labels_object = viewer.layers['training_labels_object'].data

### 3.3.3.2. Creating training data by means of vectors (polygons, lines etc.)
Please first run the code below to add a 'shapes' layer to napari. Also indicate the class to which the objects covered by the shapes should belong (second line in the code below). All the objects should belong to the same class. If you are analysing 3D data, please also indicate the layer number in which you will label the objects (the layer number can be seen in the bottom right corner of the napari viewer when working with 3D data).

In [30]:
viewer.add_shapes(name = 'shapes')
classnumber = 1
layernumber = 20

For the iterative training and classification using polygons, please carry out the procedure below.

1. Select the 'shapes' layer in napari and add shapes to it, using the 'Add polygons' icon or other instruments (see the icons in the top left corner of the viewer, under 'Layer controls'). To label an object, it is sufficient that at least one pixel belonging to the object is covered by the shape.
2. After having drawn the shapes, please run the code below, which converts the shapes to raster labels and adds them to the 'training_labels_object' layer.
3. Additional shapes can be added in napari, using the same workflow: first create a new shapes layer and indicate the class to which the objects should be assigned (code cell above), then draw the shapes, and run the code below.
4. Once all shapes have been added, please go to steps 3.3.3.3 and 3.3.3.4.
5. After running the code in 3.3.3.3 and 3.3.3.4, additional vector labels can be added in napari, and classification can be carried out again by running the code below and in steps 3.3.3.3 and 3.3.3.4, in an iterative way, until the result of the classification is satisfactory.

In [31]:
# Get shapes from napari viewer and then remove the layer in napari
shapes = viewer.layers['shapes']
viewer.layers.remove('shapes')

# Convert shapes to labels
labels = shapes.to_labels(image.shape) if nrofdim == 2 else np.zeros(image.shape, dtype=np.uint8)
if nrofdim == 3:
    labels[layernumber, :, :] = shapes.to_labels(image.shape[1:3])

# Add the labels to the 'training_labels_object' layer
training_labels_object[labels > 0] = classnumber

### 3.3.3.3. Training the classifier
After the training data has been created in steps 3.3.3.1 and/or 3.3.3.2, please run the code below to train the classifier

In [11]:
# Remove existing 'class_image' layer in the napari viewer
if 'class_image' in viewer.layers: 
    viewer.layers.remove('class_image')
yobj = training_labels_object.ravel()

# Function to calculate the median of the non-zero elements in the objects. The non-zero elements correspond to the pixels 
# where the user has drawn a label. When different labels have been drawn over an object, the median of the labelled
# pixels defines the label of the object.
def medianOfNonZeros(regionmask, training_labels_object):
    training_labels_object_mask = training_labels_object[regionmask]
    non_zero_elements = training_labels_object_mask[training_labels_object_mask != 0]
    if non_zero_elements.size == 0:
        return 0
    elif (non_zero_elements.size) % 2 == 0:
        return np.median(non_zero_elements[:-1])
    else:
        return np.median(non_zero_elements)

# Calculate properties of the objects (including the median of their non-zero pixels)
annotation_stats = regionprops(labeled_hyst_watershed, intensity_image=training_labels_object, extra_properties=(medianOfNonZeros,))
annotated_class = np.asarray([s.medianOfNonZeros for s in annotation_stats])
maskObj = annotated_class > 0

# Train the random forest object classifier
classifierObj = RandomForestClassifier(n_jobs=-1)
classifierObj.fit(Xobj[maskObj], annotated_class[maskObj])

RandomForestClassifier(n_jobs=-1)

### 3.3.3.4. Applying the classifier to the entire dataset
After the classification has been completed, the result of the classification is automatically added as the 'class_image' layer in napari. The labels can be saved by selecting the 'training_labels_object' layer and applying 'File -> Save Selected Layer(s)' in napari.
The classification can be saved by selecting the 'class_image' layer and applying 'File -> Save Selected Layer(s)'.

In [12]:
# Apply the classifier to the complete dataset
resultObj = classifierObj.predict(Xobj)

# Create an empty classification image
class_image = np.zeros(image.shape[:3], dtype=np.uint8)

# Add the classified objects to the classification image
for k, result in enumerate(resultObj):
    if result > 0:
        if nrofdim == 2:
            class_image[labeled_hyst_watershed == k + 1] = result
        else:
            startz, starty, startx, stopz, stopy, stopx = annotation_stats[k].bbox
            portion = (slice(startz, stopz), slice(starty, stopy), slice(startx, stopx))
            class_image_portion = class_image[portion]
            labeled_portion = labeled_hyst_watershed[portion]
            class_image_portion[labeled_portion == k + 1] = result
            class_image[portion] = class_image_portion

# Add classification image to the napari viewer
viewer.add_labels(class_image, opacity=0.8)

<Labels layer 'class_image' at 0x2a0a040e610>

## 3.3.4. Save classifier
To save the classifier, please specify its name below. In the default code below, it is saved in the same folder as the Jupyter Notebook. Otherwise, its path should be specified. 

In [34]:
f = open('Object_classifier.pckl', 'wb')
pickle.dump(classifierObj, f)
f.close()

# 4. Manual classification

## 4.1. Manual assignment of all objects to one class
If the vast majority of the objects belong to one class, it may be more straightforward to assign them manually to this class than to perform machine learning based object classification (section 3.3). In that case, run the code below and first adjust the class number in the first line. The objects not belonging to this class can then be assigned to their correct class in steps 4.2, 4.3 and 4.4.

In [20]:
classnumber = 1
class_image = np.zeros(image.shape[:3], dtype=np.uint8)
class_image[labeled_hyst_watershed > 0] = classnumber
viewer.add_labels(class_image, opacity = 0.8)

<Labels layer 'class_image [1]' at 0x1910fea82b0>

## 4.2. Manual object re-classification 
Objects wrongly classified by the classifier in section 3.3, or manually in section 4.1, can be manually assigned to a different class. Please first run the code below to add an empty 'manual_labels_object' labels layer to napari. Afterwards, if you would like to use raster labels for the classification, please go to step 4.2.1; if you would like to use vector shapes, please go to step 4.2.2.

In [37]:
manual_labels_object = np.zeros(image.shape[:3], dtype=np.uint8)
viewer.add_labels(manual_labels_object)

<Labels layer 'manual_labels_object' at 0x178147abd60>

### 4.2.1. By means of raster labels

For the manual object classification, please carry out the procedure below.
1. Select the 'manual_labels_object' layer in napari and add labels to it, using the paint brush and other instruments (see the icons in the top left corner of the viewer under 'Layer controls'). To label an object, it is sufficient that at least one pixel belonging to the object is labeled. 
2. After having labelled the objects, please run the code line below, and go to step 4.2.3. 

In [13]:
manual_labels_object = viewer.layers['manual_labels_object'].data

### 4.2.2. By means of vectors (polygons, lines etc.)
Please first run the code below to add a 'shapes' layer to napari. Also indicate the class to which the objects covered by the shapes should belong (second line in the code below). All the objects should belong to the same class. When working with 3D data, please also indicate the horizontal slice number in which you label the objects (the slice number can be seen in the bottom right corner of the napari viewer when working with 3D data).

In [38]:
viewer.add_shapes(name = 'shapes')
classnumber = 6
slicenumber = 6

For the manual object classification using vector labels, please carry out the procedure below. 
1. Select the 'shapes' layer in napari and add shapes to it, using the 'Add polygons' or other instruments (see the icons in the top left corner of the viewer, under 'Layer controls'). To label an object, it is sufficient that at least one pixel belonging to the object is covered by the shape.
2. After having drawn the shapes, please run the code below, which converts the shapes to raster labels and adds them to the 'manual_labels_object' layer.
3. Additional shapes can be added in napari, using the same workflow: first create a new 'shapes' layer and indicate the class to which the objects should be assigned (code cell above), then draw the shapes, and run the code below. 
4. Once all labels have been added, please go to step 4.2.3.

In [39]:
# Get shapes from napari viewer and then remove the layer in napari
shapes = viewer.layers['shapes']
viewer.layers.remove('shapes')

# Convert shapes to labels
labels = shapes.to_labels(image.shape) if nrofdim == 2 else np.zeros(image.shape, dtype=np.uint8)
if nrofdim == 3:
    labels[slicenumber, :, :] = shapes.to_labels(image.shape[1:3])

# Add the labels to the 'manual_labels_object' layer
manual_labels_object[labels > 0] = classnumber

### 4.2.3. Applying the modifications made
The modifications made in step 4.2.1 or 4.2.2 are applied to the 'class_image' layer by running the code below. If an object has received different labels, the median of the labelled pixel values determines the class number.
The labels can be saved by selecting the 'manual_labels_object' layer and applying 'File -> Save Selected Layer(s)' in napari. The classification can be saved by selecting the 'class_image' layer and applying 'File -> Save Selected Layer(s)'.

In [14]:
# Remove existing 'class_image' layer in the napari viewer
if 'class_image' in viewer.layers: 
    viewer.layers.remove('class_image') 

# Function to calculate the median of the non-zero elements in the objects. The non-zero elements correspond to the pixels 
# where the user has drawn a label. When different labels have been drawn over an object, the median of the labelled
# pixels defines the label of the object.
def medianOfNonZeros(regionmask, manual_labels_object):
    manual_labels_object_mask = manual_labels_object[regionmask]
    non_zero_elements = manual_labels_object_mask[manual_labels_object_mask != 0]
    if non_zero_elements.size == 0:
        return 0
    elif (non_zero_elements.size) % 2 == 0:
        return np.median(non_zero_elements[:-1])
    else:
        return np.median(non_zero_elements)

# Calculate properties of the objects (including the median of their non-zero pixels)
annotation_stats_man = regionprops(labeled_hyst_watershed, intensity_image=manual_labels_object, extra_properties=(medianOfNonZeros,))
annotated_class_man = np.asarray([s.medianOfNonZeros for s in annotation_stats_man])

# Add the manually re-classified objects to the classification image
for k, s in enumerate(annotation_stats_man):
    if annotated_class_man[k] > 0:
        if nrofdim == 2:
            starty, startx, stopy, stopx = s.bbox
            portion = (slice(starty, stopy), slice(startx, stopx))
        else:            
            startz, starty, startx, stopz, stopy, stopx = s.bbox
            portion = (slice(startz, stopz), slice(starty, stopy), slice(startx, stopx))
        class_image_portion = class_image[portion]
        labeled_portion = labeled_hyst_watershed[portion]
        class_image_portion[labeled_portion == k + 1] = annotated_class_man[k]
        class_image[portion] = class_image_portion

# Add the classification image to the napari viewer
viewer.add_labels(class_image, opacity=0.8)

<Labels layer 'class_image' at 0x2a0a2d32fd0>

## 4.3. Pixel-based correction of existing objects (no modification of background)
Besides the manual re-classification of entire objects (section 4.2), parts of objects can be assigned to a different class, by labelling pixels belonging to that object. Please first run the code below to add an empty 'manual_labels_pixel' layer to napari. Afterwards, if you would like to use raster labels for the classification, please go to step 4.3.1; if you would like to use vector shapes, please go to step 4.3.2.

In [44]:
manual_labels_pixel = np.zeros(image.shape[:3], dtype=np.uint8)
viewer.add_labels(manual_labels_pixel)

<Labels layer 'manual_labels_pixel' at 0x1787b66c880>

### 4.3.1. By means of raster labels

For the manual pixel-based correction, please carry out the procedure below.
1. Add labels to the 'manual_labels_pixel' layer in napari, using the paint brush and other instruments. This can be done for all existing classes. Only the pixels that belong to an already existing object, will be re-classified. Where the label is painted outside an existing object (i.e. in the background), it remains background. 
2. When the labels have been added, please run the code below, and go to step 4.3.3.

In [77]:
manual_labels_pixel = viewer.layers['manual_labels_pixel'].data

In 3D data, the correction will only apply to the horizontal slice which was annotated. If you want to apply it to all slices, please run the code below. Indicate the slice which was annotated in the first line of the code.

In [78]:
slicenumber = 6
manual_labels_pixel = np.tile(manual_labels_pixel[slicenumber, :, :], (image.shape[0], 1, 1))
viewer.add_labels(manual_labels_pixel)

<Labels layer 'manual_labels_pixel [1]' at 0x1dc1e4bf040>

### 4.3.2. By means of vector polygons
Please first run the code below to add a 'shapes' layer to napari. Also indicate the class to which the pixels covered by the shapes should belong (second line in the code below). All re-classified pixels should belong to the same class. When working with 3D data, please also indicate the slice number in which you label the objects (the slice number can be seen in the bottom right corner of the napari viewer).

In [45]:
viewer.add_shapes(name = 'shapes')
classnumber = 2
slicenumber = 6

For the manual pixel-based classification using vector labels, please carry out the procedure below.

1. Select the 'shapes' layer in napari and add shapes to it, using 'Add polygons' or other instruments (see the icons in the top left corner of the viewer, under 'Layer controls'). Only the pixels that belong to an already existing object, will be re-classified. Where the shape extends outside an existing object (i.e. in the background), the background remains unchanged.
2. After having drawn the shapes, please run the code below, which converts the shapes to raster labels and adds them to the 'manual_labels_pixel' layer.
3. For 3D data, the correction will only apply to the horizontal slice which was annotated if the variable 'applyToAllSlices' is set to 'N' (first line of the code below). If you want the correction to apply to all slices, please set this variable to 'Y'.
4. Additional shapes can be added in napari, using the same workflow: first create a new 'shapes' layer and indicate the class to which the objects should be assigned (code cell above), then draw the shapes, and run the code below.
5. Once all labels have been added, please go to step 4.3.3.

In [46]:
applyToAllSlices = 'Y' # For 3D data: apply changes to all slices ('Y') or only to the annotated slice ('N')

# Get shapes from napari viewer and then remove the layer in napari
shapes = viewer.layers['shapes']
viewer.layers.remove('shapes')

# Convert shapes to labels
labels = shapes.to_labels(image.shape) if nrofdim == 2 else np.zeros(image.shape, dtype=np.uint8)
if nrofdim == 3:
    labels_slice = shapes.to_labels(image.shape[1:3])
    if applyToAllSlices == 'N':
        labels[slicenumber,:,:] = labels_slice
    else:
        labels = np.tile(labels_slice,[image.shape[0],1,1])

# Add the labels to the 'manual_labels_pixel' layer
manual_labels_pixel[labels > 0] = classnumber

### 4.3.3. Applying the modifications made
The modifications made in step 4.3.1 or 4.3.2 are applied to the 'class_image' layer by running the code below. The labels can be saved by selecting the 'manual_labels_pixel' layer and applying 'File -> Save Selected Layer(s)' in napari. The classification can be saved by selecting the 'class_image' layer and applying 'File -> Save Selected Layer(s)'.

In [47]:
if 'class_image' in viewer.layers: 
    viewer.layers.remove('class_image')

# Add the manually corrected pixels to the classification image
class_image[manual_labels_pixel > 0] = manual_labels_pixel[manual_labels_pixel > 0]

# Any pixels labeled outside existing objects are discarded
class_image[labeled_hyst_watershed == 0] = 0

# Add the classification image to the napari viewer
viewer.add_labels(class_image, opacity = 0.65)

<Labels layer 'class_image' at 0x17851bb2250>

## 4.4. Pixel-based correction (including background), or creation of new objects
Whereas in section 4.3 pixels can be re-classified only if they belong to an existing object, in this section any pixel can be re-classified (including the background) by labelling it. In this way also new objects can be created. 
1. If you would like to use raster labels for this correction, simply add labels to the 'class_image' layer, using the paint brush and other instruments. Where the label is painted outside existing objects (i.e. in the background class), these pixels are assigned to the new class. It may be useful to save the 'class_image' layer first, by selecting that layer and applying 'File -> Save Selected Layer(s)' in napari (because changes can only be undone manually with the eraser and paint brush).
2. If you would like to use vector shapes for the correction, please first run the code below to add a 'shapes' layer to napari. Also indicate the class to which the pixels covered by the shapes should belong (second line in the code below). All re-classified pixels should belong to the same class. When working with 3D data, please also indicate the horziontal slice number in which you label the objects (the slice number can be seen in the bottom right corner of the napari viewer).

In [48]:
viewer.add_shapes(name = 'shapes')
classnumber = 6
slicenumber = 50

For the manual pixel-based correction (including background) using vector labels, please carry out the procedure below.

1. Select the 'shapes' layer in napari and add shapes to it, using the 'Add polygons' or other instruments (see the icons in the top left corner of the viewer, under 'Layer controls'). Where the shape is drawn outside existing objects (i.e. in the background class), these pixels are assigned to the new class. In this way, this tool can be used for the creation of new objects.
2. After having drawn the shapes, please run the code below, which converts the shapes to raster labels and adds them to the 'class_image' layer.
3. Additional shapes can be added in napari, using the same workflow: first create a new 'shapes' layer and indicate the class to which the objects should be assigned (code cell above), then draw the shapes, and run the code below.

In [49]:
# Get shapes from napari viewer and then remove the layer in napari
shapes = viewer.layers['shapes']
viewer.layers.remove('shapes')

# Convert shapes to labels
labels = shapes.to_labels(image.shape) if nrofdim == 2 else np.zeros(image.shape,dtype=np.uint8)
if nrofdim == 3:
    labels[slicenumber,:,:] = shapes.to_labels(image.shape[1:3])  

# Add the labels to the classification image
class_image[labels > 0] = classnumber

# 5. Postprocessing
## 5.1. Creation of 2-D image from 3-D data
When analysing 3D data, it may be useful to project the 3D segmentation onto a 2D map, e.g. for use in a GIS. To create a 2D map from a 3D label volume, please run the code below.

In [36]:
# Initialize the class_image_slice array
class_image_slice = np.zeros(image.shape[1:3], dtype=np.uint8)

for m in range(image.shape[1]):
    for n in range(image.shape[2]):
        class_image_column = class_image[:, m, n]
        non_zero_elements = class_image_column[class_image_column != 0]
        
        # Assign the values
        if non_zero_elements.size == 0:
            class_image_slice[m, n] = 0
        elif (non_zero_elements.size) % 2 == 0:
            class_image_slice[m, n] = np.median(non_zero_elements[:-1])
        else:
            class_image_slice[m, n] = np.median(non_zero_elements)

# Add labels to the viewer
viewer.add_labels(class_image_slice, opacity=0.65)

<Labels layer 'class_image_slice [2]' at 0x1c0ca8a1f40>

## 5.2. Creation of separate labels layers for each class
A text file with the class names needs to be provided. In this file, the text must be of the form: {"1": "Ditch", "2": "Road", "3": "House"}. In the default code below, this text file is named 'Classes.txt'. If it is named differently, please specify its name below. It should be in the same folder as the Jupyter Notebook, or otherwise its path should be specified in the code below.<br> Within the folder where the input data is located, a 'Classes' folder will be created (first code cell below). In that folder, for each class a file with the objects belonging to that class will be stored (second code cell below). When working with 3D data, please indicate in the first line of the second code cell below if the files representing individual classes should be 2D (i.e. extracted from the 2D map created in step 5.1) or 3D.

In [21]:
# Read the file with the class names
with open("Classes.txt","r") as file:
    classes = json.loads(file.read())

# Make a folder named 'Classes'
pathname = os.path.dirname(viewer.layers[0].source.path)
pathnameClasses = pathname + '/Classes/'
pathnameClassesExist = os.path.exists(pathnameClasses)
if not pathnameClassesExist:
    os.makedirs(pathnameClasses)   

In [22]:
filesIn3D = 'N' # For 3D data: make 2D class layers ('N') or 3D class layers ('Y') 
class_image = viewer.layers['class_image'].data
for m in range(len(classes)):
    classnr = int(list(classes)[m])
    if nrofdim == 2:
        objectclass = np.zeros(image.shape, dtype=np.uint8)
        objectclass[class_image == classnr] = classnr
    else:
        if filesIn3D == 'N':
            objectclass = np.zeros(image.shape[1:3], dtype=np.uint8)
            objectclass[class_image_slice == classnr] = classnr            
        else:
            objectclass = np.zeros(image.shape, dtype=np.uint8)
            objectclass[class_image == classnr] = classnr
    mstr = str(classnr)
    classname = classes[mstr]
    viewer.add_labels(objectclass, opacity = 0.8, name = classname)
    fullname = pathnameClasses + classname + '.tif'
    viewer.layers[classname].save(fullname)  

## 5.3. Morhological operations
With the code below, morphological operations can be performed. First, indicate the number of the class to which the operation should be applied and the radius of the structuring element. Other structuring elements can be used, see https://scikit-image.org/docs/stable/api/skimage.morphology.html. 

In [30]:
classnumber = 3 # Define the class to which the operation should be applied
classname = classes[str(classnumber)]
original = viewer.layers[classname].data

# Create structuring element
radius = 7
if nrofdim == 2:
    footprint = morphology.disk(radius)
else:
    if filesIn3D == 'N':
        footprint = morphology.disk(radius)
    else:
        footprint = morphology.ball(radius)

Four operations can be performed: morphological dilation, erosion, closing and opening. Specify the operation in the first cell below. For other operations, see https://scikit-image.org/docs/stable/api/skimage.morphology.html. The result of the operation is saved in the 'Classes' folder created in section 5.2.

In [31]:
operation = 'closing' # please specify the operation: 'dilation', 'erosion', 'opening' or 'closing'

In [32]:
# the variable 'original' can be replaced by another variable, e.g. 'processed' to use the result of a previous operation
if operation == 'dilation':
    processed = morphology.dilation(original, footprint) 
elif operation == 'erosion':
    processed = morphology.erosion(original, footprint)
elif operation == 'opening':
    processed = morphology.opening(original, footprint)
elif operation == 'closing':
    processed = morphology.closing(original, footprint)

In [33]:
# Add the result to the napari viewer and save it as a .tif file
viewer.add_labels(processed, opacity = 0.8, name = classname + '_morph_' + operation)
fullname = pathnameClasses + classname + '_morph_' + operation + '.tif'
viewer.layers[classname + '_morph_' + operation].save(fullname)

['J:/Archeologie/Falerii_Novi_2022/Publicaties/TBD/Zenodo/Classes/Linear_feature_morph_closing.tif']

# 6. Convert raster images to polygons, and save them as shapefiles
If the input image is a GeoTIFF, the following information is automatically extracted by running the code in the first cell below:
1. x-coordinate (Easting) and y-coordinate (Northing) of the lower left corner of the input data;
2. sample distance in x- and y-direction (in m);
3. coordinate reference system (EPSG code: https://epsg.org/home.html). <br>

If the input image is not a GeoTIFF an error message will be given.
The above information should then be entered manually in the second cell below.

In [ ]:
# This code is based on: https://stackoverflow.com/questions/79059686/read-xyz-geotiff-file-with-tifffile
with tifffile.TiffFile(path) as tif:
    tif_tags = {}
    for tag in tif.pages[0].tags.values():
        name, value = tag.name, tag.value
        tif_tags[name] = value
coordx = tif_tags['ModelTiepointTag'][3]
coordy = tif_tags['ModelTiepointTag'][4] - tif_tags['ImageLength'] * tif_tags['ModelPixelScaleTag'][1]
sampledistx = tif_tags['ModelPixelScaleTag'][0]
sampledisty = tif_tags['ModelPixelScaleTag'][1]
CoordRefSyst = 'EPSG:' + str(tif_tags['GeoKeyDirectoryTag'][15])
print('The following information has been extracted:\nThe Easting and Northing of the lower left corner are (' + str(coordx) + ',' + str(coordy) + ').\nThe sample distance in x and y direction is ' + str(sampledistx) + ' and ' + str(sampledisty) + ' m, respectively.\nThe EPSG is ' + str(CoordRefSyst) + '.')

In [4]:
coordx = 395750 
coordy = 4586831 
sampledistx = 0.05 
sampledisty = 0.05 
CoordRefSyst = "EPSG:32633" 

To run the code below, a text file mentioning the classes should be provided. In this file, the text should be of the form: {"1": "Ditch", "2": "Road", "3": "House"}. In the default code below, this text file is named 'Classes_test.txt'. If it is named differently, please specify its name below. It should be in the same folder as the Jupyter Notebook, or otherwise its path should be specified in the code below. <br> A subfolder called 'Shapefiles' will be created in the folder where input data is located. For each class, a shapefile with the name of the class is created. The path and file names can be changed by modifying the 'pathname' and 'fullname' variables in the code below.

In [5]:
with open("Classes.txt","r") as file:
    classes = json.loads(file.read())
pathname = os.path.dirname(viewer.layers[0].source.path)
pathnameshp = pathname + '/Shapefiles/'
pathnameshpExist = os.path.exists(pathnameshp)
if not pathnameshpExist:
    os.makedirs(pathnameshp)

In [ ]:
# This code is based on: 
# https://docs.opencv.org/3.4/d4/d73/tutorial_py_contours_begin.html
# https://docs.opencv.org/4.x/d9/d8b/tutorial_py_contours_hierarchy.html
# https://stackoverflow.com/questions/57965493/how-to-convert-numpy-arrays-obtained-from-cv2-findcontours-to-shapely-polygons
# https://gis.stackexchange.com/questions/353057/how-to-create-a-shapely-polygon-with-a-hole
# https://shapely.readthedocs.io/en/stable/reference/shapely.MultiPolygon.html
# https://shapely.readthedocs.io/en/2.0.3/manual.html

for r in range(len(classes)):
    classnr = list(classes)[r]   
    classname = classes[classnr]
    classbinarymap = viewer.layers[classname].data
    classbinarymap[classbinarymap == int(classnr)] = 1   
    classbinarymap2 = np.zeros(classbinarymap.shape, dtype=np.uint8)
    
    # Label connected components
    classbinarymaplabeled = label(classbinarymap, connectivity=1)
    
    # Compute region properties
    stats = regionprops(classbinarymaplabeled)
    
    # Filter bounding boxes
    for q, s in enumerate(stats):
        box = s.bbox
        if box[2] - box[0] > 1 and box[3] - box[1] > 1:
            classbinarymap2[classbinarymaplabeled == q + 1] = 1
    
    # Find contours
    #contours, _ = cv.findContours(classbinarymap2, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    contours, hierarchy = cv.findContours(classbinarymap2, cv.RETR_CCOMP, cv.CHAIN_APPROX_NONE)
    #contoursSq = map(np.squeeze, contours)

    # Convert contours to polygons. Take into account possible holes in polygons using the hierarchy of the contours
    listofpolygons = []
    for k in range(len(contours)):
    	if hierarchy[0][k][3] == -1:
    		exterior = []
    		holes = []
    		for m in range(len(contours[k])):
    			tuple_element = (contours[k][m][0][0],contours[k][m][0][1])
    			exterior.append(tuple_element)
    		for p in range(len(contours)):
    			if hierarchy[0][p][3] == k:
    				interior = []
    				for n in range(len(contours[p])):
    					tuple_element = (contours[p][n][0][0],contours[p][n][0][1])
    					interior.append(tuple_element)
    					interior[::-1]
    				holes.append(interior)
    		pgn = Polygon(exterior,holes)
    		listofpolygons.append(pgn)
    multipolygon = MultiPolygon(listofpolygons)
       
    # Apply geometric transformations
    multipolygonflipud1 = affine_transform(multipolygon,[1, 0, 0, -1, 0, 0])
    if nrofdim == 2:
        multipolygonflipud2 = translate(multipolygonflipud1, yoff=image.shape[0])
    elif nrofdim == 3:
        multipolygonflipud2 = translate(multipolygonflipud1, yoff=image.shape[1])
    multipolygonscaled = scale(multipolygonflipud2, xfact=sampledistx, yfact=sampledisty, origin=(0,0))    
    multipolygontransl = translate(multipolygonscaled, xoff=coordx, yoff=coordy)
    
    # Create GeoDataFrame
    gdf = gpd.GeoDataFrame(index=[0], geometry=[multipolygontransl], crs=CoordRefSyst)
    
    # Explode multipolygon into individual geometries
    gdf_exploded = gdf.explode()
    
    # Save shapefile    
    fullname = pathnameshp + classname + '.shp'
    gdf_exploded.to_file(fullname)

# 7. Tools
## 7.1. Simple thresholding
A simple threshold can be applied to the geophysical image instead of machine learning-based pixel classification and hysteresis thresholding (section 2). The threshold can be defined by the user or found using a particular method, e.g. Otsu's method (https://scikit-image.org/docs/stable/api/skimage.filters.html#skimage.filters.threshold_otsu)
After running the code below, object classification can be performed (section 3).

In [12]:
thresh = 1.75
# thresh = filters.threshold_otsu(image) 
binary = image > thresh
viewer.add_labels(binary, opacity = 0.7)
labeled_binary = label(binary, connectivity = 2, background = 0)
viewer.add_labels(labeled_binary, name = "Objects (Class 2)")

<Labels layer 'Objects (Class 2) [1]' at 0x1f18e6e2580>

## 7.2. Correcting the boundaries of magnetometer anomalies
For the correction, the horizontal gradient magnitude of the magnetometer data is calculated. For each anomaly, of the magnetometer data corresponding to the highest horizontal gradients, the mean is used as the threshold to delineate the corrected anomaly (please see the paper and the supplementaty data accompanying it). This operation needs the following input:
1. the geophysical image,
2. the labeled objects after hysteresis thresholding (section 2.8.3 or 3.1),
3. the object classification: machine learning based (section 3) or manual (section 4).

The result is a new classification map, with the boundaries of the objects corrected. 

In [26]:
class_image = viewer.layers['class_image'].data

In [ ]:
class_image_corrected = AnomalyEdge.find_edge_of_anomaly(image, labeled_hyst, class_image)
viewer.add_labels(class_image_corrected, opacity = 0.7)

## 7.3. Comparison of segmentations by means of quantitative metrics
The result of the segmentation can be compared agaist a reference or 'ground truth' by means of different metrics. See https://scikit-learn.org/stable/api/sklearn.metrics.html and https://rasbt.github.io/mlxtend

### 7.3.1. Confusion matrix
In the code below, 'y_true' is the reference classification map, 'y_pred' is the prediction made by applying the steps above.

In [ ]:
cm = confusion_matrix(y_true, y_pred)

In [ ]:
# Please replace the class names in 'class_dict' by the classes in the segmentation maps to be compared.
class_dict = {0: 'Background',
              1: 'Circular',
              2: 'Linear'}
fig, ax = plot_confusion_matrix(conf_mat=cm,colorbar=True,class_names=class_dict.values())

# The code below uses a log-normalised colourmap. 
fig, ax = plot_confusion_matrix(conf_mat=cm,norm_colormap=matplotlib.colors.LogNorm(),colorbar=True,class_names=class_dict.values())

### 7.3.2. Intersection over Union 
The Jaccard score, Jaccard index or Intersection over Union (IoU) metric is found by taking the intersection between a class in the reference classification and the same class in the segmentation map to be evaluated. It is divided by the union of the class in the reference and in the segmentation map to be evaluated. The IoU for each class can be returned (fist line of code below), or the mean IoU can be calculated, in different ways (see https://scikit-learn.org/stable/modules/generated/sklearn.metrics.jaccard_score.html)

In [ ]:
js = jaccard_score(y_true, y_pred, average=None)
js = jaccard_score(y_true, y_pred, average='macro')

### 7.3.3. F1 score
The F1 score is equal to (2 * TP)/(2 * TP + FP + FN), where TP is the number of true positives, FN is the number of false negatives, and FP is the number of false positives. The F1 score can be given for each class separately (code below), or it can be averaged in different ways (see https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)

In [ ]:
f1score = f1_score(y_true, y_pred, average=None)

## 7.4. Send data from this notebook to napari viewer, and vice versa

In [20]:
# Import data from a layer in the napari viewer into this notebook
importeddata = viewer.layers['LayerToBeImported'].data # change the names as needed

# Display data in napari viewer
# Display Labels or segmentation
viewer.add_labels(dataToBeDisplayed, opacity = 0.7, name = 'DisplayedLayer') # change names and opacity as needed

# Display an image
viewer.add_image(imageToBeDisplayed, opacity = 0.7, name = 'DisplayedImage') # change names and opacity as needed

<Image layer 'DisplayedImage' at 0x1c0c5ab7340>